# Forecasting Net Prophet

You’re a growth analyst at [MercadoLibre](http://investor.mercadolibre.com/about-us). With over 200 million users, MercadoLibre is the most popular e-commerce site in Latin America. You've been tasked with analyzing the company's financial and user data in clever ways to make the company grow. So, you want to find out if the ability to predict search traffic can translate into the ability to successfully trade the stock.

The instructions for this Challenge are divided into four steps, as follows:

* Step 1: Find unusual patterns in hourly Google search traffic

* Step 2: Mine the search traffic data for seasonality

* Step 3: Relate the search traffic to stock price patterns

* Step 4: Create a time series model with Prophet

The following subsections detail these steps.

## Install and import the required libraries and dependencies

In [ ]:
# Install the required libraries
#!pip install prophet

In [ ]:
# Import the required libraries and dependencies
import pandas as pd
from prophet import Prophet
import datetime as dt
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime,timedelta
import warnings

%matplotlib inline

## Step 1: Find Unusual Patterns in Hourly Google Search Traffic

The data science manager asks if the Google search traffic for the company links to any financial events at the company. Or, does the search traffic data just present random noise? To answer this question, pick out any unusual patterns in the Google search data for the company, and connect them to the corporate financial events.

To do so, complete the following steps:

1. Read the search data into a DataFrame, and then slice the data to just the month of May 2020. (During this month, MercadoLibre released its quarterly financial results.) Visualize the results. Do any unusual patterns exist?

2. Calculate the total search traffic for the month, and then compare the value to the monthly median across all months. Did the Google search traffic increase during the month that MercadoLibre released its financial results?


#### Step 1: Read the search data into a DataFrame, and then slice the data to just the month of May 2020. (During this month, MercadoLibre released its quarterly financial results.) Visualize the results. Do any unusual patterns exist?

In [ ]:
# Store the data in a Pandas DataFrame
# Set the "Date" column as the Datetime Index.
warnings.filterwarnings("ignore")
df_mercado_trends = pd.read_csv(
    "https://static.bc-edx.com/ai/ail-v-1-0/m8/lms/datasets/google_hourly_search_trends.csv",
    index_col='Date',
    parse_dates=True,
    infer_datetime_format=True
).dropna()

# Review the first and last five rows of the DataFrame
display(df_mercado_trends.head())
display(df_mercado_trends.tail())

In [ ]:
# Review the data types of the DataFrame using the info function
df_mercado_trends.info()

In [ ]:
# Slice the DataFrame to just the month of May 2020
# Plot to visualize the data for May 2020
# Create a line plot
# Adjust the figure size if needed
df_mercado_trends[(df_mercado_trends.index.month == 5) &  (df_mercado_trends.index.year == 2020)].\
    plot(figsize=(10, 4),title = "May 2020 Mercado Search Trend")
plt.grid(True, linestyle='--', linewidth=0.5, color='gray', alpha=0.5)
plt.ylabel('Search')

#### Step 2: Calculate the total search traffic for the month, and then compare the value to the monthly median across all months. Did the Google search traffic increase during the month that MercadoLibre released its financial results?

In [ ]:
# Calculate the sum of the total search traffic for May 2020
traffic_may_2020  =  df_mercado_trends[(df_mercado_trends.index.month == 5) & \
                                        (df_mercado_trends.index.year == 2020)]\
                                            .resample('M').sum()

# View the traffic_may_2020 value
print(f"Total search traffic for May 2020: {traffic_may_2020.iloc[0,0]}")
print('-'*100)


In [ ]:
# Calcluate the monhtly median search traffic across all months 
# Group the DataFrame by index year and then index month, chain the sum and then the median functions
median_monthly_traffic  = df_mercado_trends.groupby([df_mercado_trends.index.year,df_mercado_trends.index.month])\
    .agg({'Search Trends':'sum'}).median()
# View the median_monthly_traffic value
print(f"Monhtly median search traffic across all months: {median_monthly_traffic['Search Trends']}")
print('-'*100)


In [ ]:
# To compare the seach traffic for the month of May 2020 to the overall monthly median value,
# Divide the may traffic total by the median monthly traffic
May_2020_to_overall_monthly_median = round(traffic_may_2020.iloc[0,0]/median_monthly_traffic['Search Trends'],2)
print(f"Seach traffic for the month of May 2020 to the overall monthly median value:{May_2020_to_overall_monthly_median}")
print('-'*100)

In [ ]:
df_mercado_trends

##### Answer the following question: 

**Question:** Did the Google search traffic increase during the month that MercadoLibre released its financial results?

**Answer:** Monthly median search traffic across all months 35172.5 is less than the total May 2020 search traffic,
 hence search traffic increase during the month that MercadoLibre released its financial results by a factor .09.

## Step 2: Mine the Search Traffic Data for Seasonality

Marketing realizes that they can use the hourly search data, too. If they can track and predict interest in the company and its platform for any time of day, they can focus their marketing efforts around the times that have the most traffic. This will get a greater return on investment (ROI) from their marketing budget.

To that end, you want to mine the search traffic data for predictable seasonal patterns of interest in the company. To do so, complete the following steps:

1. Group the hourly search data to plot the average traffic by the hour of day. Does the search traffic peak at a particular time of day or is it relatively consistent?

2. Group the hourly search data to plot the average traffic by the day of the week (for example, Monday vs. Friday). Does the search traffic get busiest on any particular day of the week?

3. Group the hourly search data to plot the average traffic by the week of the year. Does the search traffic tend to increase during the winter holiday period (weeks 40 through 52)?


#### Step 1: Group the hourly search data to plot the average traffic by the hour of the day.

In [ ]:
# Group the hourly search data to plot the average traffic by the day of week 
average_traffic = df_mercado_trends.groupby([df_mercado_trends.index.hour])\
    .agg({'Search Trends':'mean'})
# Create a line plot
# Adjust the figure size if needed
average_traffic.plot(figsize=(10, 6), title = "Average search traffic by the day of a month")
plt.xlabel('Day')
plt.ylabel('Search')

#### Step 2: Group the hourly search data to plot the average traffic by the day of the week (for example, Monday vs. Friday).

In [ ]:
# Group the hourly search data to plot the average traffic by the day of week 
weekly_avg_traffic = df_mercado_trends.groupby(df_mercado_trends.index.weekday).agg({'Search Trends':'mean'})
# Create a line plot
weekly_avg_traffic.plot(figsize=(10, 6))
plt.title("Average search traffic by day of week")
plt.xlabel('Day')
plt.ylabel('Search')

#### Step 3: Group the hourly search data to plot the average traffic by the week of the year.

In [ ]:
# Group the hourly search data to plot the average traffic by the week of the year 
yearly_avg_traffic = df_mercado_trends.groupby(df_mercado_trends.index.isocalendar().week).agg({'Search Trends':'mean'})
# Create a line plot
# Adjust the figure size if needed
yearly_avg_traffic.plot(figsize=(10, 6))
plt.title("Average search traffic by day of week numbers")
plt.xlabel('week')
plt.ylabel('Search')


##### Answer the following question:

**Question:** Are there any time based trends that you can see in the data?

**Answer:**  There is time based trend during the first and last three weeks of the year, it might be due to holidays.

## Step 3: Relate the Search Traffic to Stock Price Patterns

You mention your work on the search traffic data during a meeting with people in the finance group at the company. They want to know if any relationship between the search data and the company stock price exists, and they ask if you can investigate.

To do so, complete the following steps:

1. Read in and plot the stock price data. Concatenate the stock price data to the search data in a single DataFrame.

2. Market events emerged during the year of 2020 that many companies found difficult. But, after the initial shock to global financial markets, new customers and revenue increased for e-commerce platforms. Slice the data to just the first half of 2020 (`2020-01` to `2020-06` in the DataFrame), and then plot the data. Do both time series indicate a common trend that’s consistent with this narrative?

3. Create a new column in the DataFrame named “Lagged Search Trends” that offsets, or shifts, the search traffic by one hour. Create two additional columns:

    * “Stock Volatility”, which holds an exponentially weighted four-hour rolling average of the company’s stock volatility

    * “Hourly Stock Return”, which holds the percent change of the company's stock price on an hourly basis

4. Review the time series correlation, and then answer the following question: Does a predictable relationship exist between the lagged search traffic and the stock volatility or between the lagged search traffic and the stock price returns?


#### Step 1: Read in and plot the stock price data. Concatenate the stock price data to the search data in a single DataFrame.

In [ ]:
# Upload the "mercado_stock_price.csv" file into Colab, then store in a Pandas DataFrame
# Set the "date" column as the Datetime Index.
warnings.filterwarnings("ignore")
df_mercado_stock = pd.read_csv(
    "https://static.bc-edx.com/ai/ail-v-1-0/m8/lms/datasets/mercado_stock_price.csv",
    index_col="date",
    parse_dates=True, 
    infer_datetime_format=True
).dropna()

# View the first and last five rows of the DataFrame
display(df_mercado_stock.head())
display(df_mercado_stock.tail())

In [ ]:
# Visualize the closing price of the df_mercado_stock DataFrame
df_mercado_stock.plot(figsize=(10, 6),title = "Mercado Stock closing price trend")
plt.ylabel("Closing price")
plt.xlabel('Date')

In [ ]:
# Concatenate the df_mercado_stock DataFrame with the df_mercado_trends DataFrame
# Concatenate the DataFrame by columns (axis=1), and drop and rows with only one column of data

try:
    df_mercado_search_stock = pd.concat([df_mercado_trends,df_mercado_stock], axis=1)[['Search Trends','close']].dropna(how ='any')
except NameError as n:
    print(f"No data frame name {n}")

# View the first and last five rows of the DataFrame
display(df_mercado_search_stock.head())
display(df_mercado_search_stock.tail())

#### Step 2: Market events emerged during the year of 2020 that many companies found difficult. But, after the initial shock to global financial markets, new customers and revenue increased for e-commerce platforms. Slice the data to just the first half of 2020 (`2020-01` to `2020-06` in the DataFrame), and then plot the data. Do both time series indicate a common trend that’s consistent with this narrative?

In [ ]:
# For the combined dataframe, slice to just the first half of 2020 (2020-01 through 2020-06) 
first_half_2020 = df_mercado_search_stock.loc['2020-01-01':'2020-06-30']
# View the first and last five rows of first_half_2020 DataFrame
display(first_half_2020.head())
display(first_half_2020.tail())

In [ ]:
# Visualize the close and Search Trends data
# Plot each column on a separate axes using the following syntax
# `plot(subplots=True)`
first_half_2020.plot(subplots=True, layout=(len(first_half_2020.columns), 1), figsize=(10, 6))
plt.show()


##### Answer the following question:

**Question:** Do both time series indicate a common trend that’s consistent with this narrative?

**Answer:** In truth, our evidence of a common trend between search activity and stock price is pretty limited. It does appear generally however that the low point for Mercado Libre's stock price, late March to early April, did coincide with lower search activity for the firm. Search activity does pick up along with its stock price, although the stock price from mid-April onward definitively outpaces search interest.

There's one important thing to note: if you zoom in both graphs to the spike around May 5th, 2020, you'll find the date of the company's earnings release to the public. It looks like search traffic for the firm spiked about two hours before the firm's stock price spiked. This could indicate a predictable pattern (can Google Searches predict the stock price?), but an alternative explanation might just be that people tend to search for more information about the firm right before an earnings announcement gets released. We'd need to do a little more time-series research to see which explanation best fits the data.

#### Step 3: Create a new column in the DataFrame named “Lagged Search Trends” that offsets, or shifts, the search traffic by one hour. Create two additional columns:

* “Stock Volatility”, which holds an exponentially weighted four-hour rolling average of the company’s stock volatility

* “Hourly Stock Return”, which holds the percent change of the company's stock price on an hourly basis


In [ ]:
# Create a new column in the mercado_stock_trends_df DataFrame called Lagged Search Trends
# This column should shift the Search Trends information by one hour
df_mercado_search_stock['Lagged Search Trends'] = df_mercado_search_stock.index + timedelta(hours = 1)

df_mercado_search_stock.head()

In [ ]:
# Create a new column in the mercado_stock_trends_df DataFrame called Stock Volatility
# This column should calculate the standard deviation of the closing stock price return data over a 4 period rolling window
df_mercado_search_stock['Stock Volatility'] = df_mercado_search_stock['close'].pct_change().rolling(window=4).std()

df_mercado_search_stock.head()


In [ ]:
# Visualize the stock volatility
df_mercado_search_stock['Stock Volatility'].plot(figsize=(10, 6))
plt.xlabel('date')
plt.ylabel('volatility')

**Solution Note:** Note how volatility spiked, and tended to stay high, during the first half of 2020. This is a common characteristic of volatility in stock returns worldwide: high volatility days tend to be followed by yet more high volatility days. When it rains, it pours.

In [ ]:
# Create a new column in the mercado_stock_trends_df DataFrame called Hourly Stock Return
# This column should calculate hourly return percentage of the closing price
df_mercado_search_stock['Hourly Stock Return'] = df_mercado_search_stock['close'].pct_change()

In [ ]:
# View the first and last five rows of the mercado_stock_trends_df DataFrame
display(df_mercado_search_stock.head())
display(df_mercado_search_stock.tail())

#### Step 4: Review the time series correlation, and then answer the following question: Does a predictable relationship exist between the lagged search traffic and the stock volatility or between the lagged search traffic and the stock price returns?

In [ ]:
# Construct correlation table of Stock Volatility, Lagged Search Trends, and Hourly Stock Return

round(df_mercado_search_stock.corr(method='pearson'),2)


##### Answer the following question:


**Question:** Does a predictable relationship exist between the lagged search traffic and the stock volatility or between the lagged search traffic and the stock price returns?

**Answer:** If we read down the second column (or read along the second row), we see that there's a slight negative correlation between searches for the firm and its subsequent stock volatility. More searches (slightly) tend to indicate less near-term hourly stock risk for the firm.

On the other hand, there's a positive correlation between search activity in one hour and stock returns in the next: as search activity goes up, so will the firm's stock price in the short term. This effect is very weak though, only half the size of the effect on stock volatility. With the correlation so close to zero, some might also say that there's almost no relationship between these two variables whatsoever, and that the small findings that we do have could be just due to chance. While there's no required threshold when looking at correlation between stock returns and the variables used to predict them, it does tend to be that correlations in this context tend to be very low across the board. It's hard to predict the market!

## Step 4: Create a Time Series Model with Prophet

Now, you need to produce a time series model that analyzes and forecasts patterns in the hourly search data. To do so, complete the following steps:

1. Set up the Google search data for a Prophet forecasting model.

2. After estimating the model, plot the forecast. How's the near-term forecast for the popularity of MercadoLibre?

3. Plot the individual time series components of the model to answer the following questions:

    * What time of day exhibits the greatest popularity?

    * Which day of the week gets the most search traffic?

    * What's the lowest point for search traffic in the calendar year?


#### Step 1: Set up the Google search data for a Prophet forecasting model.

In [ ]:
# Using the df_mercado_trends DataFrame, reset the index so the date information is no longer the index

prophet_df = df_mercado_trends.reset_index()

# Label the columns ds and y so that the syntax is recognized by Prophet
prophet_df.rename(columns={'Date':'ds','Search Trends':'y'}, inplace=True)

# Drop an NaN values from the prophet_df DataFrame
prophet_df.dropna(how = 'any',inplace=True)

# View the first and last five rows of the mercado_prophet_df DataFrame
display(prophet_df.head())
display(prophet_df.tail())

In [ ]:
# Call the Prophet function, store as an object
model = Prophet()

In [ ]:
# Fit the time-series model.
model.fit(prophet_df)

In [ ]:
# Create a future dataframe to hold predictions
# Make the prediction go out as far as 2000 hours (approx 80 days)
future_dataframe = model.make_future_dataframe(periods=2000,
                                     freq='h')

# View the last five rows of the future_mercado_trends DataFrame
future_dataframe.head()

In [ ]:
# Make the predictions for the trend data using the future_mercado_trends DataFrame
forecast_mercado_trends = model.predict(future_dataframe)


# Display the first five rows of the forecast_mercado_trends DataFrame
forecast_mercado_trends.head()

#### Step 2: After estimating the model, plot the forecast. How's the near-term forecast for the popularity of MercadoLibre?

In [ ]:
# Plot the Prophet predictions for the Mercado trends data
warnings.filterwarnings("ignore")
model.plot(forecast_mercado_trends.reset_index(),figsize= (12,6))


##### Answer the following question:

**Question:**  How's the near-term forecast for the popularity of MercadoLibre?

**Answer:** 


#### Step 3: Plot the individual time series components of the model to answer the following questions:

* What time of day exhibits the greatest popularity?

* Which day of the week gets the most search traffic?

* What's the lowest point for search traffic in the calendar year?


In [ ]:
# Set the index in the forecast_mercado_trends DataFrame to the ds datetime column
try:
    forecast_mercado_trends.set_index('ds', inplace=True)
except KeyError as e:
    print(e)


In [ ]:
# View the only the yhat,yhat_lower and yhat_upper columns from the DataFrame
forecast_mercado_trends_filter = forecast_mercado_trends[['yhat','yhat_lower','yhat_upper']]
forecast_mercado_trends_filter

Solutions Note: `yhat` represents the most likely (average) forecast, whereas `yhat_lower` and `yhat_upper` represents the worst and best case prediction (based on what are known as 95% confidence intervals).

In [ ]:
# From the forecast_mercado_trends DataFrame, plot the data to visualize
#  the yhat, yhat_lower, and yhat_upper columns over the last 2000 hours 

# Filter the last 2000 hours
recent_data = forecast_mercado_trends_filter.tail(2000)
recent_data.plot(figsize=(12,6))



In [ ]:
# Reset the index in the forecast_mercado_trends DataFrame
# Use the plot_components function to visualize the forecast results 
# for the forecast_canada DataFrame 
warnings.filterwarnings("ignore")
model.plot_components(forecast_mercado_trends.reset_index(),figsize=(12,12))
plt.show()



##### Answer the following questions:

**Question:** What time of day exhibits the greatest popularity?

**Answer:** 

**Question:** Which day of week gets the most search traffic? 
   
**Answer:** 

**Question:** What's the lowest point for search traffic in the calendar year?

**Answer:** 
